In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('../data/Advertising.csv')

#### In this part we will import the different regression models to compare the performance for each of them (simple linear model, ridge regression and lasso regression)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [4]:
data_x = data.drop('sales', axis = 1)
data_y = data['sales']

#### For the sake of this exercise, we will transform the dataset with polynomial features

In [5]:
poly_transformer = PolynomialFeatures(degree = 3, include_bias = False)

data_x = pd.DataFrame(poly_transformer.fit_transform(data_x),columns = poly_transformer.get_feature_names_out())

In [6]:
#Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.3, random_state=101)

#### In this part we will perform the feature scaling (standardization), to keep all the features with the same scale and prevent any of them to impact the predictions more than the other ones, and also, to be able to compare the features each other. Keep in mind that the scaler should be fitted in the X_train set ONLY, in order to prevent data leakage

In [7]:
scaler = StandardScaler()

df_cols = data_x.columns
X_train = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns = df_cols)

X_test = scaler.transform(X_test)
X_test = pd.DataFrame(X_test, columns = df_cols)

#### Let's define the following functions in order to modularize the modeling steps

In [8]:
def fit_model(model,X_train,y_train):
    model_object = model.fit(X_train,y_train)
    return model_object
def use_model(model, X_test):
    predictions = model.predict(X_test)
    return predictions
def evaluate_model(metric,predictions,y_test):
    performance = metric(y_test, predictions)
    return performance

In [9]:
#Let's define the linear models
lin_regr = LinearRegression()
ridge_regr = Ridge(alpha = 10)
lasso_regr = Lasso(alpha = 10)
models = [lin_regr, ridge_regr, lasso_regr]

In [10]:
resulting_models = []
for model in models:
    resulting_models.append(fit_model(model, X_train, y_train))

In [11]:
predictions = []
for model in resulting_models:
    predictions.append(use_model(model, X_test))

In [12]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [13]:
metrics = []
for prediction in predictions:
    mae = evaluate_model(mean_absolute_error, prediction, y_test)
    rmse = np.sqrt(evaluate_model(mean_squared_error, prediction, y_test))
    metrics.append((mae, rmse))

In [14]:
metrics

[(0.41275160852975196, 0.5803286825159616),
 (0.5774404204714175, 0.8946386461319681),
 (4.618428571428571, 5.399973733874139)]

#### So far we've tested linear regression vs ridge regression vs lasso regression, with a lambda value of 10 (this is given arbitrarily). The conclusion here is that the model performs better without regularization due to the lower RMSE corresponds to the linear regression model (in this case multiple linear regression).

#### In theory this is not overfitting because the performance in both data sets is pretty well, and that's why the regularization methods lead to an increase in variance an then a lower performance.

#### Now we will use cross validation to determine the best lambda for both regularization methods. 

In [30]:
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV

#### We will call the get_scorer_names module from sklearn.metrics in order to obtain the name of the scorers useful for the cross-validation process and then choose any of them

In [16]:
from sklearn.metrics import get_scorer_names

In [17]:
get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_

In [18]:
#In this case we've chosen negative mean squared error
ridge_cv = RidgeCV(alphas = (0.1, 1, 10), scoring = 'neg_mean_squared_error')

In [19]:
ridge_cv.fit(X_train, y_train)
predictions_ridge_cv = ridge_cv.predict(X_test)

In [20]:
ridge_cv.alpha_

0.1

In [21]:
np.sqrt(mean_squared_error(y_test, predictions_ridge_cv))

0.6180719926948917

In [22]:
mean_absolute_error(y_test, predictions_ridge_cv)

0.42737748843414897

#### By modelling the Ridge regression algorithm with cross-validation, we obtained that the best lambda among 0.1, 1 and 10 is 0.1, because this resulted in a higher negative mean squared error and therefore, a better performance than before when we implemented the ridge regression with lambda = 10 (0.61 with lambda = 0.1 against 0.89 with lambda = 10).

</br>

#### In this last part we are going to do the same with the lasso regression model

In [23]:
lasso_cv = LassoCV(eps = 0.01, n_alphas = 100, max_iter = 10000, cv = 5)
lasso_cv.fit(X_train, y_train)

LassoCV(cv=5, eps=0.01, max_iter=10000)

In [24]:
lasso_cv.alpha_

0.04943070909225832

In [25]:
predictions_lasso_cv = lasso_cv.predict(X_test)

In [26]:
np.sqrt(mean_squared_error(y_test, predictions_lasso_cv))

0.7615440303260841

In [27]:
mean_absolute_error(y_test, predictions_lasso_cv)

0.5159788188265407

#### By looking at the metrics, we can see that the Lasso regression model performance improved a lot with respect to the previous model, which has a lambda  = 10 (RMSE = 5.39, lambda = 10 and RMSE = 0.76, lambda = 0.049).

</br>

#### Finally we will look at the coefficients. We can see that some of them are 0 and this is the difference with respect to ridge regression; in this case, even though the Lasso regression had a lower performance than ridge, it performed feature selection and left only the relevant variables which the huge difference between both models.

In [28]:
lasso_cv.coef_

array([ 2.35600233,  0.21183181,  0.        , -0.        ,  3.78675114,
       -0.        ,  0.        ,  0.0624318 ,  0.        , -1.01152151,
       -0.        , -0.        ,  0.        ,  0.        , -0.        ,
        0.        ,  0.        ,  0.        ,  0.        ])

In [29]:
ridge_cv.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])

### Now let's do the same for ElasticNet regression

In [75]:
el_net_cv = ElasticNetCV(l1_ratio = np.arange(0.1,1.1,0.1), eps = 0.001, n_alphas = 100, max_iter = 10000)

In [76]:
el_net_cv.fit(X_train, y_train)

ElasticNetCV(l1_ratio=array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
             max_iter=10000)

In [77]:
el_net_cv.alpha_

0.004943070909225833

In [78]:
el_net_cv.l1_ratio_

1.0

In [79]:
elastic_predictions = el_net_cv.predict(X_test)

In [80]:
np.sqrt(mean_squared_error(y_test, elastic_predictions))

0.6063140748984046

In [81]:
mean_absolute_error(y_test, elastic_predictions)

0.4335034618590077

#### We can see now that the ElasticNet regression improved the model performance, even more than Lasso regression did. by combining every parameter; in l1_ratio, it tested values between 0 and 1, with a path length of 0.1, and a lambda value from 0.001 until 100 with a path length of 0.001. 
</br>

#### Given that the nature of ElasticNet regression is integrating both regularization, we could think of this function as an 'automatic' way of finding the best regularization method and its parameters. In our case it selected an l1_ratio = 1, which means that the best model is actually l1. Otherwise, it would have adjusted a model with the better alpha (lambda) for both regularization terms in the equation an their respective weights (l1_ratio)